### Load the pretrained model

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

In [2]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" 

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

c:\Users\abdel\.conda\envs\ML\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abdel\.cache\huggingface\hub\models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

# Try chatting with the model

In [21]:
chat = [
    {"role": "system", "content": "You are an expert in IA and you assist people with building their IA systems and fine tuning large language models based on their needs."},
    {"role": "user", "content": "Hey, what is the best way to fine tune a large language model based on my data, which is a set of research papers?"}
]

In [22]:
formatted_chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
print("Formatted chat:\n", formatted_chat)

Formatted chat:
 <｜begin▁of▁sentence｜>You are an expert in IA and you assist people with building their IA systems and fine tuning large language models based on their needs.<｜User｜>Hey, what is the best way to fine tune a large language model based on my data, which is a set of research papers?<｜Assistant｜>


In [23]:
inputs = tokenizer(formatted_chat, return_tensors="pt", add_special_tokens=False)

In [24]:
inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}

In [25]:
outputs = model.generate(**inputs, max_new_tokens=2048, temperature=0.1)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [26]:
decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
print("Decoded output:\n", decoded_output)

Decoded output:
 <think>
Okay, so I need to figure out the best way to fine tune a large language model based on my data, which is a set of research papers. I'm not entirely sure where to start, but I know that fine tuning involves adjusting the model to better fit the data I have. Let me break this down step by step.

First, I should probably understand what exactly fine tuning is. From what I remember, it's about making the model more accurate by tweaking its parameters based on the data it's trained on. So, if I have a bunch of research papers, I need to see how the model performs on those documents and adjust it accordingly.

I think the first thing I should do is assess how well the model currently is performing. Maybe I can look at some metrics like accuracy, BLEU score, or other relevant evaluation metrics. But wait, how do I get accurate metrics from research papers? I guess I need to parse the papers and extract relevant information that the model can understand. That might be

# Fine Tuning the Model

In [27]:
dataset = load_dataset("../cache/training_dataset")

NameError: name 'load_dataset' is not defined

In [ ]:
training_args = TrainingArguments(
    output_dir="../cache/fine_tuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="../cache/logs",
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()